In [14]:
# imports
import pandas as pd
import statistics
import numpy as np
from datetime import datetime

In [15]:
# read the data (2012)
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

# read the data (2017)
# df = pd.read_csv('data/BPI Challenge 2017-training.csv')

# read the data (2018)
# df = pd.read_csv('data/BPI Challenge 2018-training.csv')

# read the data (road traffic fines)
# df = pd.read_csv('data/Road_Traffic_Fine_Management_Process-training.csv')

df.columns


Index(['eventID ', 'case concept:name', 'case REG_DATE', 'case AMOUNT_REQ',
       'event concept:name', 'event lifecycle:transition',
       'event time:timestamp'],
      dtype='object')

In [16]:
df.head(10)

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
5,4294967297,173691,2011-10-01T08:08:58.256+02:00,5000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:09:02.195
6,4294967298,173691,2011-10-01T08:08:58.256+02:00,5000,A_PREACCEPTED,COMPLETE,01-10-2011 08:09:56.648
7,4294967299,173691,2011-10-01T08:08:58.256+02:00,5000,W_Completeren aanvraag,SCHEDULE,01-10-2011 08:09:59.578
8,8589934592,173694,2011-10-01T08:10:30.287+02:00,7000,A_SUBMITTED,COMPLETE,01-10-2011 08:10:30.287
9,8589934593,173694,2011-10-01T08:10:30.287+02:00,7000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 08:10:30.591


In [17]:
case_column = "case concept:name"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
position_column = "Position"
predicted_next_event_column = "Prediction for Next Activity"
predicted_next_timestamp_column = "Prediction for Next Timestamp"

In [18]:
df[timestamp_column] = pd.to_datetime(df[timestamp_column])
timeformat = "%Y-%m-%d %H:%M%S" # new time format
df = df.sort_values(by=[case_column, timestamp_column])  # sort values by user and time of event
df = df.reset_index()
df[position_column] = 1  # add a new column for the number of the event in the process
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214377 entries, 0 to 214376
Data columns (total 9 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   index                       214377 non-null  int64         
 1   eventID                     214377 non-null  int64         
 2   case concept:name           214377 non-null  int64         
 3   case REG_DATE               214377 non-null  object        
 4   case AMOUNT_REQ             214377 non-null  int64         
 5   event concept:name          214377 non-null  object        
 6   event lifecycle:transition  214377 non-null  object        
 7   event time:timestamp        214377 non-null  datetime64[ns]
 8   Position                    214377 non-null  int64         
dtypes: datetime64[ns](1), int64(5), object(3)
memory usage: 14.7+ MB


In [19]:
# create a dictionary lenght_process that stores the lenght of all processes indexed by the case number
# fill in the column created earlier that shows which position is a certain event in the process (the first event is 1)
lenght_process = {}
step_number = 1
for row in df.index:
    if row == 0:
        step_number = 1
    else:
        if df[case_column][row] == df[case_column][row - 1]:
            step_number += 1
        else:
            lenght_process[df[case_column][row]] = step_number
            step_number = 1
    df[position_column][row] = step_number

#df[position_column] = df.groupby([case_column]).cumcount()+1
#lenght_process = {x: df[df[case_column]==x][position_column].max() for x in df[case_column]}

df[df[case_column] == 173688]

C:\Users\Sander\AppData\Local\Temp/ipykernel_11308/505159642.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[position_column][row] = step_number


,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,1
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,2
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,3
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,4
4,89,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,5
5,94,5,173688,2011-10-01T00:38:44.546+02:00,20000,A_ACCEPTED,COMPLETE,2011-01-10 11:42:43.308,6
6,98,7,173688,2011-10-01T00:38:44.546+02:00,20000,A_FINALIZED,COMPLETE,2011-01-10 11:45:09.243,7
7,99,6,173688,2011-10-01T00:38:44.546+02:00,20000,O_SELECTED,COMPLETE,2011-01-10 11:45:09.243,8
8,100,8,173688,2011-10-01T00:38:44.546+02:00,20000,O_CREATED,COMPLETE,2011-01-10 11:45:11.197,9
9,101,9,173688,2011-10-01T00:38:44.546+02:00,20000,O_SENT,COMPLETE,2011-01-10 11:45:11.380,10


In [20]:
# here you can see different statistics about the process lenght
print(f'Mean: {statistics.mean(lenght_process.values())}')
print(f'Mode: {statistics.mode(lenght_process.values())}')
print(f'Median: {statistics.median(lenght_process.values())}')
print(f'Max: {max(lenght_process.values())}') # this is df[df['case concept:name'] == 185548]
print(f'Min: {min(lenght_process.values())}')

Mean: 20.478983568972104
Mode: 3
Median: 11.0
Max: 175
Min: 3


In [21]:
# make a function that determines the most common value in a list
def most_common(list):
    return max(set(list), key = list.count)

In [22]:
# the list prediction_step stores the most common event for each position
# for example, the most common activity for position 1 is prediction_step[0]; prediction for position n is prediction_step[n-1]
prediction_step = []
for i in range(1, max(lenght_process.values()) + 1):
    prediction_step.append(most_common([df[event_column][row] for row in df[df[position_column] == i].index]))
prediction_step

['A_SUBMITTED',
 'A_PARTLYSUBMITTED',
 'W_Afhandelen leads',
 'W_Afhandelen leads',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Completeren aanvraag',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offertes',
 'W_Nabellen offert

In [23]:
# We add the predicted next event to the dataframe

df[predicted_next_event_column] = df[position_column].apply(lambda x: prediction_step[x - 1]) 
df[df[case_column] == 173688]

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,Prediction for Next Activity
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,1,A_SUBMITTED
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,2,A_PARTLYSUBMITTED
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,3,W_Afhandelen leads
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,4,W_Afhandelen leads
4,89,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,5,W_Completeren aanvraag
5,94,5,173688,2011-10-01T00:38:44.546+02:00,20000,A_ACCEPTED,COMPLETE,2011-01-10 11:42:43.308,6,W_Completeren aanvraag
6,98,7,173688,2011-10-01T00:38:44.546+02:00,20000,A_FINALIZED,COMPLETE,2011-01-10 11:45:09.243,7,W_Completeren aanvraag
7,99,6,173688,2011-10-01T00:38:44.546+02:00,20000,O_SELECTED,COMPLETE,2011-01-10 11:45:09.243,8,W_Completeren aanvraag
8,100,8,173688,2011-10-01T00:38:44.546+02:00,20000,O_CREATED,COMPLETE,2011-01-10 11:45:11.197,9,W_Completeren aanvraag
9,101,9,173688,2011-10-01T00:38:44.546+02:00,20000,O_SENT,COMPLETE,2011-01-10 11:45:11.380,10,W_Completeren aanvraag


In [24]:
df[timestamp_column][0]

Timestamp('2011-01-10 00:38:44.546000')

In [25]:
average_time_per_pos = {}
number_of_adds = {}
for i in range(1, max(lenght_process.values()) + 1):
    average_time_per_pos[i] = pd.Timedelta(0)
    number_of_adds[i] = 0

for row in df.index:
    if ((row+1) < len(df) and df[case_column][row] == df[case_column][row+1]):
        average_time_per_pos[df[position_column][row]] += (df[timestamp_column][row+1] - df[timestamp_column][row])
        number_of_adds[df[position_column][row]] += 1

average_time_per_pos = {x: (average_time_per_pos[x] / number_of_adds[x] if number_of_adds[x]!=0 else pd.Timedelta(0)) for x in average_time_per_pos.keys() & number_of_adds}
#average_time_per_pos

In [26]:
# We create a dictionary to store the average time duration until the next event
# For example the average time duration between position n and n+1 is average_time_per_pos[n]
# The dataframe must be sorted on case-number and timestamp for this calculation!

average_time_per_pos = {}
number_of_adds = {}
for i in range(1, max(lenght_process.values()) + 1):
    average_time_per_pos[i] = pd.Timedelta(0)
    number_of_adds[i] = 0

for row in df.index:
    if ((row+1) < len(df) and df[case_column][row] == df[case_column][row+1]):
        average_time_per_pos[df[position_column][row]] += (df[timestamp_column][row+1] - df[timestamp_column][row])
        number_of_adds[df[position_column][row]] += 1

average_time_per_pos = {x: (average_time_per_pos[x] / number_of_adds[x] if number_of_adds[x]!=0 else pd.Timedelta(0)) for x in average_time_per_pos.keys() & number_of_adds}
#average_time_per_pos

In [27]:
# We add the predicted time duration until next event to the dataframe

def next_timestamp(x):
    return (x[timestamp_column] + average_time_per_pos[x[position_column]]) if (average_time_per_pos[x[position_column]] != pd.Timedelta(0)) else "-"
df[predicted_next_timestamp_column] = df.apply(next_timestamp, axis=1)
df[df[case_column] == 173688]

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,Prediction for Next Activity,Prediction for Next Timestamp
0,0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,2011-01-10 00:38:44.546,1,A_SUBMITTED,2011-01-10 01:02:15.692799980
1,1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,2011-01-10 00:38:44.880,2,A_PARTLYSUBMITTED,2011-01-14 17:34:06.019567962
2,2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,2011-01-10 00:39:37.906,3,W_Afhandelen leads,2011-01-12 15:42:45.133977503
3,3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,2011-01-10 00:39:38.875,4,W_Afhandelen leads,2011-01-18 17:58:08.134373441
4,89,4,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,START,2011-01-10 11:36:46.437,5,W_Completeren aanvraag,2011-01-11 01:44:00.864780820
5,94,5,173688,2011-10-01T00:38:44.546+02:00,20000,A_ACCEPTED,COMPLETE,2011-01-10 11:42:43.308,6,W_Completeren aanvraag,2011-01-17 15:48:59.507239409
6,98,7,173688,2011-10-01T00:38:44.546+02:00,20000,A_FINALIZED,COMPLETE,2011-01-10 11:45:09.243,7,W_Completeren aanvraag,2011-01-11 18:33:31.521018354
7,99,6,173688,2011-10-01T00:38:44.546+02:00,20000,O_SELECTED,COMPLETE,2011-01-10 11:45:09.243,8,W_Completeren aanvraag,2011-01-15 17:58:18.506804690
8,100,8,173688,2011-10-01T00:38:44.546+02:00,20000,O_CREATED,COMPLETE,2011-01-10 11:45:11.197,9,W_Completeren aanvraag,2011-01-12 17:33:36.758614116
9,101,9,173688,2011-10-01T00:38:44.546+02:00,20000,O_SENT,COMPLETE,2011-01-10 11:45:11.380,10,W_Completeren aanvraag,2011-01-15 14:47:24.342350575


In [28]:
df[df[predicted_next_timestamp_column] == "-"]

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,Prediction for Next Activity,Prediction for Next Timestamp
78609,154402,16333760626854,185548,2011-11-15T13:42:45.592+01:00,20000,W_Nabellen incomplete dossiers,COMPLETE,2012-10-01 19:48:49.515,175,W_Nabellen incomplete dossiers,-
